In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# Load the data

In [98]:
import datetime
from helpers import *

height, weight, gender = load_data(sub_sample=True, add_outlier=True)
x, mean_x, std_x = standardize(height)
y, tx = build_model_data(x, weight)

In [99]:
y.shape, tx.shape

((202,), (202, 2))

# Computing the Cost Function
Fill in the the `compute_cost` function below:

In [100]:
def compute_loss(y, tx, w):
    e = y - tx.dot(w)
    """MAE:"""
    return (1 /y.shape[0]) * np.sum(np.abs(e))
    """
    MSE:
    return (1 /(2 * y.shape[0])) * e.T.dot(e)
    """

compute_loss(y, tx, [-1, 2])

75.067805854926391

# Grid Search

Fill in the function `grid_search()` below:

In [101]:
def grid_search(y, tx, w0, w1):
    losses = np.zeros((len(w0), len(w1)))
    for i in range(len(w0)):
        for j in range(len(w1)):
            losses[i,j] = compute_loss(y, tx, [w0[i], w1[j]])
    return losses

Let us play with the grid search demo now!

In [102]:
from grid_search import generate_w, get_best_parameters
from plots import grid_visualization
import datetime

# Generate the grid of parameters to be swept
grid_w0, grid_w1 = generate_w(num_intervals=200)

# Start the grid search
start_time = datetime.datetime.now()
grid_losses = grid_search(y, tx, grid_w0, grid_w1)

# Select the best combinaison
loss_star, w0_star, w1_star = get_best_parameters(grid_w0, grid_w1, grid_losses)
end_time = datetime.datetime.now()
execution_time = (end_time - start_time).total_seconds()

# Print the results
print("Grid Search: loss*={l}, w0*={w0}, w1*={w1}, execution time={t:.3f} seconds".format(
      l=loss_star, w0=w0_star, w1=w1_star, t=execution_time))

# Plot the results
fig = grid_visualization(grid_losses, grid_w0, grid_w1, mean_x, std_x, height, weight)
fig.set_size_inches(10.0,6.0)
fig.savefig("grid_plot")  # Optional saving

Grid Search: loss*=5.350259618178881, w0*=73.36683417085428, w1*=15.829145728643226, execution time=0.922 seconds


/home/hugo/anaconda3/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


# Gradient Descent

Again, please fill in the functions `compute_gradient` below:

In [112]:
def compute_gradient(y, tx, w):
    c = -1 / len(y)
    e = y - tx.dot(w)
    """MAE subgradient:"""
    return c * tx.T.dot(np.sign(e))
    """
    MSE gradient:
    return c * tx.T.dot(e)
    """

print(compute_gradient(y, tx, np.array([73, 13])))

[-0.03960396 -0.35988335]


Please fill in the functions `gradient_descent` below:

In [107]:
def gradient_descent(y, tx, initial_w, max_iters, gamma): 
    """Gradient descent algorithm."""
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        gradient = compute_gradient(y, tx, w)
        print(gradient)
        loss = compute_loss(y, tx, w)
        w = w - gamma * gradient
        # store w and loss
        ws.append(np.copy(w))
        losses.append(loss)
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))

    return losses, ws

Test your gradient descent function through gradient descent demo shown below:

In [113]:
from plots import gradient_descent_visualization

# Define the parameters of the algorithm.
max_iters = 50
gamma = 10

# Initialization
w_initial = np.array([0.0, 1.0])

# Start gradient descent.
start_time = datetime.datetime.now()
gradient_losses, gradient_ws = gradient_descent(y, tx, w_initial, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Gradient Descent: execution time={t:.3f} seconds".format(t=exection_time))

[ -1.00000000e+00  -8.72789190e-16]
Gradient Descent(0/49): loss=74.06780585492638, w0=10.0, w1=1.0000000000000087
[ -1.00000000e+00  -8.72789190e-16]
Gradient Descent(1/49): loss=64.06780585492639, w0=20.0, w1=1.0000000000000173
[ -1.00000000e+00  -8.72789190e-16]
Gradient Descent(2/49): loss=54.06780585492637, w0=30.0, w1=1.000000000000026
[ -1.00000000e+00  -8.72789190e-16]
Gradient Descent(3/49): loss=44.06780585492638, w0=40.0, w1=1.0000000000000346
[ -1.00000000e+00  -8.72789190e-16]
Gradient Descent(4/49): loss=34.06780585492638, w0=50.0, w1=1.0000000000000433
[-0.96039604 -0.05279159]
Gradient Descent(5/49): loss=24.10684024142588, w0=59.603960396039604, w1=1.5279159082749174
[-0.56435644 -0.42123203]
Gradient Descent(6/49): loss=16.1852019076765, w0=65.24752475247524, w1=5.740236175158014
[-0.3960396  -0.49721844]
Gradient Descent(7/49): loss=11.473162608658766, w0=69.20792079207921, w1=10.712420563337902
[-0.28712871 -0.46971169]
Gradient Descent(8/49): loss=7.599521218256737

In [114]:
# Time Visualization
from ipywidgets import IntSlider, interact
def plot_figure(n_iter):
    fig = gradient_descent_visualization(
        gradient_losses, gradient_ws, grid_losses, grid_w0, grid_w1, mean_x, std_x, height, weight, n_iter)
    fig.set_size_inches(10.0, 6.0)

interact(plot_figure, n_iter=IntSlider(min=1, max=len(gradient_ws)))

/home/hugo/anaconda3/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


<function __main__.plot_figure>

# Stochastic gradient descent

In [61]:
def compute_stoch_gradient(y, tx, w):
    N = len(y)
    idx = np.random.randint(0, N)
    g = tx[idx] * (tx[idx].dot(w) - y[idx])
    return g

print(compute_stoch_gradient(y,tx,[90, 10]))

def stochastic_gradient_descent(
        y, tx, initial_w, batch_size, max_epochs, gamma):
    """Stochastic gradient descent algorithm."""
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_epochs):
        rgrads = [compute_stoch_gradient(y, tx, w) for i in range(batch_size)]
        gradient = np.mean(rgrads, axis = 0)
        loss = compute_loss(y, tx, w)
        w = w - gamma * gradient
        # store w and loss
        ws.append(np.copy(w))
        losses.append(loss)
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
        
    return losses, ws

[ 14.07088101  10.9054746 ]


In [63]:
# Define the parameters of the algorithm.
max_iters = 50
gamma = 0.4
batch_size = 4

# Initialization
w_initial = np.array([0.0, 0.0])

# Start SGD.
start_time = datetime.datetime.now()
gradient_losses, gradient_ws = stochastic_gradient_descent(
    y, tx, w_initial, batch_size, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("SGD: execution time={t:.3f} seconds".format(t=exection_time))

Gradient Descent(0/49): loss=279223671275.91675, w0=28.186148378569506, w1=-6.82471915841589
Gradient Descent(1/49): loss=123887647967.0529, w0=52.75091747837005, w1=17.958303336292737
Gradient Descent(2/49): loss=23642229353.05508, w0=59.80977315542564, w1=16.356617540729776
Gradient Descent(3/49): loss=11043531442.225271, w0=63.73036424913114, w1=18.42565343445732
Gradient Descent(4/49): loss=7334787250.117506, w0=66.78391286468522, w1=20.389208661757014
Gradient Descent(5/49): loss=6044656640.733556, w0=63.86132695037122, w1=15.997648002148441
Gradient Descent(6/49): loss=6304281233.401065, w0=69.28478394758442, w1=16.53093607499316
Gradient Descent(7/49): loss=2807746468.9592695, w0=70.63836738470668, w1=16.31328885516076
Gradient Descent(8/49): loss=2292645069.629924, w0=72.66309466596572, w1=14.957877722099411
Gradient Descent(9/49): loss=1667734574.0848844, w0=73.21467865639988, w1=13.959579947630012
Gradient Descent(10/49): loss=1550416403.75928, w0=71.39940744695953, w1=12.505

In [64]:
# Time Visualization
from ipywidgets import IntSlider, interact
def plot_figure(n_iter):
    fig = gradient_descent_visualization(
        gradient_losses, gradient_ws, grid_losses, grid_w0, grid_w1, mean_x, std_x, height, weight, n_iter)
    fig.set_size_inches(10.0, 6.0)

interact(plot_figure, n_iter=IntSlider(min=1, max=len(gradient_ws)))

/home/hugo/anaconda3/lib/python3.5/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


<function __main__.plot_figure>